# Data Exploration

## Explore The Data: What Data Are We Using?

We are using the World University Rankings 2020 dataset for our Universe bot.
This dataset contains information about 1395 universities from 92 countries.
Using the information about these universities, the challenge is to build a model to recommend the top universities that fit the user, based on the following fields:

- **Rank_Char** (object) - Ranking of the university according to The Times Higher Education
- **Score_Rank** (int64) - Ranking of the university according to the `Score_Result`
- **University** (object) - Name of the university
- **Country** (object) - Country where the university is located
- **Number_students** (object) - Number of students enrolled in the university
- **Numb_students_per_Staff** (float64) - Ratio between the number of students and the university staff
- **International_Students** (object) - Percentage of international students
- **Percentage_Female** (object) - Percentage of female students
- **Percentage_Male** (object) - Percentage of male students
- **Teaching** (float64) - Score in teaching
- **Research** (float64) - Score in reasearch
- **Citations** (float64) - Score in citations
- **Industry_Income** (float64) - Score in industry income
- **International_Outlook** (float64) - Score in international outlook
- **Score_Result** (float64) - Resulting score calculated by taking into account 30% of `Teaching`, 30% of `Research`, 30% of `Citations`, 7.5% of `International_Outlook` and 2.5% of `Industry_Income`)
- **Overall_Ranking** (object) - Overall ranking of the university according to The Times Higher Education

For our recommender system, we will use a content-based system and we will take into account : `Number_students`, `Teaching`, `Research`, `Citations`, `Industry_income` and `International_Outlook`.

## Import libraries

In [3]:
from google.colab import files
import pandas as pd
import numpy as np
import math
import sklearn.metrics.pairwise as pw
from google.colab import auth
import gspread
from oauth2client.client import GoogleCredentials

## Read in Data

In [4]:
uploaded = files.upload()

Saving Word_University_Rank_2020.csv to Word_University_Rank_2020.csv


In [5]:
university = pd.read_csv('Word_University_Rank_2020.csv', encoding="UTF-8")

In [6]:
df = pd.DataFrame(university)

In [7]:
df.head()

,Rank_Char,Score_Rank,University,Country,Number_students,Numb_students_per_Staff,International_Students,Percentage_Female,Percentage_Male,Teaching,Research,Citations,Industry_Income,International_Outlook,Score_Result,Overall_Ranking
0,1,1,University of Oxford,United Kingdom,"20,664",11.2,41%,46%,54%,90.5,99.6,98.4,65.5,96.4,95.4,95.40
1,2,2,California Institute of Technology,United States,"2,240",6.4,30%,34%,66%,92.1,97.2,97.9,88.0,82.5,94.5,94.50
2,3,3,University of Cambridge,United Kingdom,"18,978",10.9,37%,47%,53%,91.4,98.7,95.8,59.3,95.0,94.4,94.40
3,4,4,Stanford University,United States,"16,135",7.3,23%,43%,57%,92.8,96.4,99.9,66.2,79.5,94.3,94.30
4,5,5,Massachusetts Institute of Technology,United States,"11,247",8.6,34%,39%,61%,90.5,92.4,99.5,86.9,89.0,93.6,93.60


In [8]:
df.dtypes

Rank_Char                   object
Score_Rank                   int64
University                  object
Country                     object
Number_students             object
Numb_students_per_Staff    float64
International_Students      object
Percentage_Female           object
Percentage_Male             object
Teaching                   float64
Research                   float64
Citations                  float64
Industry_Income            float64
International_Outlook      float64
Score_Result               float64
Overall_Ranking             object
dtype: object

In [9]:
df.size

22336

In [10]:
df['University'].nunique()

1395

In [11]:
df['Country'].nunique()

92

We need to exploit the number of students, so we first replace the comma with a dot in order to change the type of the variable to float.

In [12]:
df['Number_students'] = df['Number_students'].apply(lambda x: x.replace(',','.'))

In [13]:
df['Number_students'] = df['Number_students'].astype(float, errors = 'raise')

In [14]:
df.dtypes

Rank_Char                   object
Score_Rank                   int64
University                  object
Country                     object
Number_students            float64
Numb_students_per_Staff    float64
International_Students      object
Percentage_Female           object
Percentage_Male             object
Teaching                   float64
Research                   float64
Citations                  float64
Industry_Income            float64
International_Outlook      float64
Score_Result               float64
Overall_Ranking             object
dtype: object

In [15]:
df.head()

,Rank_Char,Score_Rank,University,Country,Number_students,Numb_students_per_Staff,International_Students,Percentage_Female,Percentage_Male,Teaching,Research,Citations,Industry_Income,International_Outlook,Score_Result,Overall_Ranking
0,1,1,University of Oxford,United Kingdom,20.664,11.2,41%,46%,54%,90.5,99.6,98.4,65.5,96.4,95.4,95.40
1,2,2,California Institute of Technology,United States,2.240,6.4,30%,34%,66%,92.1,97.2,97.9,88.0,82.5,94.5,94.50
2,3,3,University of Cambridge,United Kingdom,18.978,10.9,37%,47%,53%,91.4,98.7,95.8,59.3,95.0,94.4,94.40
3,4,4,Stanford University,United States,16.135,7.3,23%,43%,57%,92.8,96.4,99.9,66.2,79.5,94.3,94.30
4,5,5,Massachusetts Institute of Technology,United States,11.247,8.6,34%,39%,61%,90.5,92.4,99.5,86.9,89.0,93.6,93.60


## Recommander System


We first perform normalization to rescale our data.

In [16]:
def normalization(df):
    df['Number_students'] = df['Number_students']/np.sqrt(df['Number_students'])
    df['Numb_students_per_Staff'] = df['Numb_students_per_Staff']/np.sqrt(df['Numb_students_per_Staff'])
    df.iloc[:,9:15] = df.iloc[:,9:15]/np.sqrt(df.iloc[:,9:15])
    return df

In [17]:
df = normalization(df)

In [18]:
df

,Rank_Char,Score_Rank,University,Country,Number_students,Numb_students_per_Staff,International_Students,Percentage_Female,Percentage_Male,Teaching,Research,Citations,Industry_Income,International_Outlook,Score_Result,Overall_Ranking
0,1,1,University of Oxford,United Kingdom,4.545767,3.346640,41%,46%,54%,9.513149,9.979980,9.919677,8.093207,9.818350,9.767292,95.40
1,2,2,California Institute of Technology,United States,1.496663,2.529822,30%,34%,66%,9.596874,9.859006,9.894443,9.380832,9.082951,9.721111,94.50
2,3,3,University of Cambridge,United Kingdom,4.356375,3.301515,37%,47%,53%,9.560335,9.934787,9.787747,7.700649,9.746794,9.715966,94.40
3,4,4,Stanford University,United States,4.016840,2.701851,23%,43%,57%,9.633276,9.818350,9.994999,8.136338,8.916277,9.710819,94.30
4,5,5,Massachusetts Institute of Technology,United States,3.353655,2.932576,34%,39%,61%,9.513149,9.612492,9.974969,9.322017,9.433981,9.674709,93.60
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1391,1001+,531,Cear� State University,Brazil,4.491548,4.219005,0%,55%,45%,4.024922,2.880972,2.469818,5.966574,4.037326,3.361547,10.7�22.1
1392,1001+,532,Pontifical Catholic University of Minas Gerais,Brazil,5.839606,4.615192,0%,57%,43%,3.911521,2.774887,2.588436,5.924525,4.404543,3.346640,10.7�22.1
1393,1001+,533,Bashkir State University,Russian Federation,3.903332,3.331666,3%,63%,37%,4.123106,2.846050,2.121320,6.074537,4.086563,3.331666,10.7�22.1
1394,1001+,534,University of Caxias do Sul,Brazil,4.135336,4.989990,0%,54%,46%,3.714835,2.792848,2.756810,5.916080,4.207137,3.316625,10.7�22.1


We drop the columns that are useless to our analysis.

In [19]:
df = df.drop('Rank_Char', axis = 1)

In [20]:
df = df.drop('Overall_Ranking', axis = 1)

In [21]:
df.head()

,Score_Rank,University,Country,Number_students,Numb_students_per_Staff,International_Students,Percentage_Female,Percentage_Male,Teaching,Research,Citations,Industry_Income,International_Outlook,Score_Result
0,1,University of Oxford,United Kingdom,4.545767,3.346640,41%,46%,54%,9.513149,9.979980,9.919677,8.093207,9.818350,9.767292
1,2,California Institute of Technology,United States,1.496663,2.529822,30%,34%,66%,9.596874,9.859006,9.894443,9.380832,9.082951,9.721111
2,3,University of Cambridge,United Kingdom,4.356375,3.301515,37%,47%,53%,9.560335,9.934787,9.787747,7.700649,9.746794,9.715966
3,4,Stanford University,United States,4.016840,2.701851,23%,43%,57%,9.633276,9.818350,9.994999,8.136338,8.916277,9.710819
4,5,Massachusetts Institute of Technology,United States,3.353655,2.932576,34%,39%,61%,9.513149,9.612492,9.974969,9.322017,9.433981,9.674709


We round our float variables to 5 decimals to keep the analysis easy.

In [23]:
df['Number_students'] = round(df['Number_students'], 5)
df['Numb_students_per_Staff'] = round(df['Numb_students_per_Staff'], 5)
df['Teaching'] = round(df['Teaching'], 5)
df['Research'] = round(df['Research'], 5)
df['Citations'] = round(df['Citations'], 5)
df['Industry_Income'] = round(df['Industry_Income'], 5)
df['International_Outlook'] = round(df['International_Outlook'], 5)

In [24]:
df.head()

,Score_Rank,University,Country,Number_students,Numb_students_per_Staff,International_Students,Percentage_Female,Percentage_Male,Teaching,Research,Citations,Industry_Income,International_Outlook,Score_Result
0,1,University of Oxford,United Kingdom,4.54577,3.34664,41%,46%,54%,9.51315,9.97998,9.91968,8.09321,9.81835,9.767292
1,2,California Institute of Technology,United States,1.49666,2.52982,30%,34%,66%,9.59687,9.85901,9.89444,9.38083,9.08295,9.721111
2,3,University of Cambridge,United Kingdom,4.35637,3.30151,37%,47%,53%,9.56033,9.93479,9.78775,7.70065,9.74679,9.715966
3,4,Stanford University,United States,4.01684,2.70185,23%,43%,57%,9.63328,9.81835,9.99500,8.13634,8.91628,9.710819
4,5,Massachusetts Institute of Technology,United States,3.35365,2.93258,34%,39%,61%,9.51315,9.61249,9.97497,9.32202,9.43398,9.674709


We create our prediction function that will get the first three universities based on the parameters we need to build our system with. In this function, we calculate the similarity between our content items with the cosine similarity method. We only call for the first parameter `Number_students` because the cosine similarity function takes the entire row into account.

In [63]:
def prediction(nb,t,r,c,i,o, df_pred):
    u = df_pred[df_pred['Number_students'] == nb]
    score = [0,0,0]
    univ_rank = [0,0,0]
    for univ in df_pred['Number_students'] :
        if univ !=nb:
            U = df_pred[df_pred['Number_students'] == univ]
            if (pw.cosine_similarity(u,U)[0][0] > score[0]):
                score[2] = score[1]
                univ_rank[2] = univ_rank[1]
                score[1] = score[0]
                univ_rank[1] = univ_rank[0]
                score[0] = float(pw.cosine_similarity(u,U)[0][0])
                univ_rank[0] = univ
            elif (pw.cosine_similarity(u,U)[0][0] > score[1])  :
                score[2] = score[1]
                univ_rank[2] = univ_rank[1]
                score[1] = float(pw.cosine_similarity(u,U)[0][0])
                univ_rank[1] = univ
            elif (pw.cosine_similarity(u,U)[0][0] > score[2]):
                score[2] = float(pw.cosine_similarity(u,U)[0][0])
                univ_rank[2] = univ
    return univ_rank

In [26]:
df.dtypes

Score_Rank                   int64
University                  object
Country                     object
Number_students            float64
Numb_students_per_Staff    float64
International_Students      object
Percentage_Female           object
Percentage_Male             object
Teaching                   float64
Research                   float64
Citations                  float64
Industry_Income            float64
International_Outlook      float64
Score_Result               float64
dtype: object

In [56]:
df_pred = df.copy()

In order to make our predictions, we need to drop all the columns that do not match our parameters. Thus, we use a copy of df that we call df_pred.

In [57]:
df_pred = df_pred.drop(df.iloc[:,0:3], axis = 1)
df_pred = df_pred.drop(df.iloc[:,4:8], axis = 1)
df_pred = df_pred.drop(df.iloc[:,13:], axis = 1)

In [58]:
df_pred.head()

,Number_students,Teaching,Research,Citations,Industry_Income,International_Outlook
0,4.54577,9.51315,9.97998,9.91968,8.09321,9.81835
1,1.49666,9.59687,9.85901,9.89444,9.38083,9.08295
2,4.35637,9.56033,9.93479,9.78775,7.70065,9.74679
3,4.01684,9.63328,9.81835,9.99500,8.13634,8.91628
4,3.35365,9.51315,9.61249,9.97497,9.32202,9.43398


In [64]:
df_pred = df_pred.append({'Number_students': 4.78000, 'Teaching': 10, 'Research': 10, 'Citations': 10, 'Industry_Income': 10, 'International_Outlook': 10}, ignore_index=True)

In [65]:
df_pred

,Number_students,Teaching,Research,Citations,Industry_Income,International_Outlook
0,4.54577,9.51315,9.97998,9.91968,8.09321,9.81835
1,1.49666,9.59687,9.85901,9.89444,9.38083,9.08295
2,4.35637,9.56033,9.93479,9.78775,7.70065,9.74679
3,4.01684,9.63328,9.81835,9.99500,8.13634,8.91628
4,3.35365,9.51315,9.61249,9.97497,9.32202,9.43398
...,...,...,...,...,...,...
1392,5.83961,3.91152,2.77489,2.58844,5.92453,4.40454
1393,3.90333,4.12311,2.84605,2.12132,6.07454,4.08656
1394,4.13534,3.71484,2.79285,2.75681,5.91608,4.20714
1395,3.04713,3.89872,3.04959,1.84391,6.29285,4.20714


In [36]:
pw.cosine_similarity(df_pred[df_pred['Number_students'] == 4.78000], df_pred[df_pred['Number_students'] == 1.49666])[0][0]

0.9896176292795601

In [37]:
result = prediction(4.78,10,10,10,10,10)

In [38]:
df[df['Number_students'] == 4.84644]

,Score_Rank,University,Country,Number_students,Numb_students_per_Staff,International_Students,Percentage_Female,Percentage_Male,Teaching,Research,Citations,Industry_Income,International_Outlook,Score_Result
199,132,George Washington University,United States,4.84644,3.61939,18%,60%,40%,6.97137,5.6921,9.02774,5.91608,7.44983,7.334848


In [39]:
df[df['Number_students'] == 6.56323]

,Score_Rank,University,Country,Number_students,Numb_students_per_Staff,International_Students,Percentage_Female,Percentage_Male,Teaching,Research,Citations,Industry_Income,International_Outlook,Score_Result
82,67,Sorbonne University,France,6.56323,3.67423,19%,58%,42%,7.89303,7.11337,8.93868,6.19677,8.31865,8.0


In [40]:
df[df['Number_students'] == 3.70027]

,Score_Rank,University,Country,Number_students,Numb_students_per_Staff,International_Students,Percentage_Female,Percentage_Male,Teaching,Research,Citations,Industry_Income,International_Outlook,Score_Result
488,265,Bournemouth University,United Kingdom,3.70027,4.13521,18%,55%,45%,5.76194,4.32435,7.44983,5.89915,8.93308,6.252999


We then create a function to display the results : we obtain the top 3 universities that match best the criterias of the user of Universe.

In [68]:
def results(result):
    L = []
    for i in result:
        L.append(df[df['Number_students'] == i].iloc[0,1])
    return L

In [42]:
results()

['Johns Hopkins University',
 'Massachusetts Institute of Technology',
 'Imperial College London']

Thanks to a Google authentication, we can get the results of the Google Forms (the user of Universe will have answered the questions) in a dataframe. We can then obtain the specific recommendations to the user.

In [43]:
auth.authenticate_user()

gc = gspread.authorize(GoogleCredentials.get_application_default())
worksheet = gc.open('UNIVERSE (réponses)').sheet1

# get_all_values gives a list of rows.
rows = worksheet.get_all_values()
print(rows)

# Convert to a DataFrame and render.
df_result = pd.DataFrame.from_records(rows)

[['Horodateur', 'Number of students', 'Score in teaching', 'Score in research', 'Score in citations', 'Score in industry income', 'Score in international outlook'], ['23/03/2021 15:37:14', '5000', '90', '80', '70', '30', '10']]


In [44]:
df_result

,0,1,2,3,4,5,6
0,Horodateur,Number of students,Score in teaching,Score in research,Score in citations,Score in industry income,Score in international outlook
1,23/03/2021 15:37:14,5000,90,80,70,30,10


In [71]:
int(df_result.iloc[1,6])

10

We write a function that our bot will call to give the recommandations to the user. We first normalize the data in the dataset from the forms. We then append our new data to df_pred. Finally, we call prediction() on our new row et give the results, which will be displayed by Universe on the Discord server.

In [66]:
def recommandation():
  #normalization
  nb = int(df_result.iloc[1,1]) / np.sqrt(int(df_result.iloc[1,1]))
  t = int(df_result.iloc[1,2]) / np.sqrt(int(df_result.iloc[1,2]))
  r = int(df_result.iloc[1,3]) / np.sqrt(int(df_result.iloc[1,3]))
  c = int(df_result.iloc[1,4]) / np.sqrt(int(df_result.iloc[1,4]))
  i = int(df_result.iloc[1,5]) / np.sqrt(int(df_result.iloc[1,5]))
  o = int(df_result.iloc[1,6]) / np.sqrt(int(df_result.iloc[1,6]))
  df_res = df_pred.append({'Number_students': nb, 'Teaching': t, 'Research': r, 'Citations': c, 'Industry_Income': i, 'International_Outlook': o}, ignore_index=True)
  #prediction
  result = prediction(nb,t,r,c,i,o, df_res)
  recommandations = results(result)
  return recommandations

In [69]:
recommandation()

['Hyogo College of Medicine',
 'St Marianna University School of Medicine',
 'Nippon Medical School']

We also write a delete() function to remove the data in the df_result dataset. Thus, any time a user wants a recommandation and fill the forms, we only get the latest data.

In [77]:
def delete():
  df_result.drop(labels = 1, inplace = True)

In [78]:
df_result

,0,1,2,3,4,5,6
0,Horodateur,Number of students,Score in teaching,Score in research,Score in citations,Score in industry income,Score in international outlook
1,23/03/2021 15:37:14,5000,90,80,70,30,10


In [79]:
delete()

In [81]:
df_result

,0,1,2,3,4,5,6
0,Horodateur,Number of students,Score in teaching,Score in research,Score in citations,Score in industry income,Score in international outlook
